In [7]:
import pandas as pd

from vcub_keeper.config import *
from vcub_keeper.reader.reader import *
from vcub_keeper.reader.reader_utils import filter_periode
from vcub_keeper.production.data import get_data_from_api_by_station, transform_json_station_data_to_df
from vcub_keeper.transform.features_factory import *

from eurybia import SmartDrift

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

%load_ext autoreload
%autoreload 2

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Objectifs 

- Analyser s'il y a un changement d'activité entre 2019 et la période du Covid.
- Analyser s'il y a un changement d'activité entre 2019 et une périodes plus normal (cf. après septembre 2021).
- Prendre en compte qu'il y a des stations où il y a peu d'activité (non monitoré par les algo).

# Période du Covid

In [8]:
# Obtenir les id des stations
station_profile = read_station_profile(path_directory=ROOT_DATA_REF)

In [11]:
# Get data from API
start_date='2019-01-01' # Année normale
stop_date='2020-12-31' # Année Covid


# list des station vcub
list_station_id = station_profile['station_id'].unique()

# +/- 13 min de query API
station_json = get_data_from_api_by_station(station_id=list_station_id, 
                                            start_date=start_date,
                                            stop_date=stop_date)

station_df = transform_json_station_data_to_df(station_json)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [76]:
station_df = get_consecutive_no_transactions_out(station_df)

In [77]:
station_df.shape

(18830394, 11)

In [78]:
station_df.head()

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,date_day,year,consecutive_no_transactions_out
0,1,2019-01-02 11:50:00,18.0,14.0,1.0,0.0,0.0,0.0,2019-01-02,2019,0
1,1,2019-01-02 12:00:00,18.0,14.0,1.0,0.0,0.0,0.0,2019-01-02,2019,1
2,1,2019-01-02 12:10:00,18.0,14.0,1.0,0.0,0.0,0.0,2019-01-02,2019,2
3,1,2019-01-02 12:20:00,14.0,18.0,1.0,4.0,0.0,4.0,2019-01-02,2019,3
4,1,2019-01-02 12:30:00,14.0,18.0,1.0,0.0,0.0,0.0,2019-01-02,2019,4


In [79]:
station_df.date.dt.year.value_counts()

2020    9563739
2019    9266655
Name: date, dtype: int64

Afin de comparer une année "normale" d'activité Vcub (Cf 2019) avec une année Covid (2020), on compare par jours et par station Vcub leurs activités.

Pour respecter la temporalité, on ne prend que les données qui sont supérieures au 17 mars (date du 1er confinement en France 17 mars 2020)

In [149]:
station_df['date_day'] = station_df['date'].dt.date

In [85]:
# transactions_in : on sommes les dépôts de Vcub à la journée
# transactions_out : on sommes les prise de Vcub à la journée
# consecutive_no_transactions_out : on prend le maximum de temps (10min) ou aucun prise de Vcub n'a été faite durant la journée.
station_agg_day_df = \
    station_df.groupby(['date_day', 'station_id'], as_index=False)[['transactions_in', 
                                                                    'transactions_out',
                                                                    'consecutive_no_transactions_out']].agg({'transactions_in': 'sum',
                                                                                                             'transactions_out': 'sum',
                                                                                                             'consecutive_no_transactions_out': 'max'})

station_agg_day_df['date_day'] = station_agg_day_df['date_day'].astype(str)
station_agg_day_df['year'] = station_agg_day_df['date_day'].apply(lambda x: x[0:4])

# Création de features temporelle
station_agg_day_df['month'] = pd.to_numeric(station_agg_day_df['date_day'].apply(lambda x: x[5:7]))

In [86]:
station_agg_day_df

,date_day,station_id,transactions_in,transactions_out,consecutive_no_transactions_out,year,month
0,2019-01-02,1,35.0,32.0,15,2019,1
1,2019-01-02,2,23.0,15.0,16,2019,1
2,2019-01-02,3,19.0,16.0,26,2019,1
3,2019-01-02,4,16.0,30.0,14,2019,1
4,2019-01-02,6,11.0,18.0,11,2019,1
...,...,...,...,...,...,...,...
131031,2020-12-31,180,0.0,0.0,931,2020,12
131032,2020-12-31,181,0.0,0.0,187,2020,12
131033,2020-12-31,182,0.0,0.0,186,2020,12
131034,2020-12-31,183,0.0,0.0,163,2020,12


In [87]:
baseline_df = station_agg_day_df[station_agg_day_df['year'] == "2019"]    # Notre année normale afin de comparer l'activité des station Vcub
covid_year_df = station_agg_day_df[station_agg_day_df['year'] == "2020"]  # Année Covid

# On respecte la même temporalité pour 2 DataFrame
baseline_df = baseline_df[baseline_df['date_day'] >= "2019-03-17"]
covid_year_df = covid_year_df[covid_year_df['date_day'] >= "2020-03-17"]


In [88]:
covid_year_df.head()

,date_day,station_id,transactions_in,transactions_out,consecutive_no_transactions_out,year,month
78077,2020-03-17,1,26.0,14.0,71,2020,3
78078,2020-03-17,2,8.0,13.0,51,2020,3
78079,2020-03-17,3,9.0,6.0,56,2020,3
78080,2020-03-17,4,12.0,12.0,10,2020,3
78081,2020-03-17,5,14.0,18.0,57,2020,3


In [89]:
covid_year_df[covid_year_df['date_day'] >= "2020-03-17"].head()

,date_day,station_id,transactions_in,transactions_out,consecutive_no_transactions_out,year,month
78077,2020-03-17,1,26.0,14.0,71,2020,3
78078,2020-03-17,2,8.0,13.0,51,2020,3
78079,2020-03-17,3,9.0,6.0,56,2020,3
78080,2020-03-17,4,12.0,12.0,10,2020,3
78081,2020-03-17,5,14.0,18.0,57,2020,3


In [93]:
FEATURE_TO_ANALYSE = ['transactions_in', 'transactions_out', 'consecutive_no_transactions_out', 'station_id', 'month']

SD = SmartDrift(df_current=covid_year_df[FEATURE_TO_ANALYSE],
                df_baseline=baseline_df[FEATURE_TO_ANALYSE],
                )

In [94]:
SD.compile(full_validation=True,
           date_compile_auc = '01/01/2020',       # Optional: useful when computing the drift for a time that is not now
           datadrift_file = "active_covid.csv"    # Optional: name of the csv file that contains the performance history of data drift
            )

The computed AUC on the X_test used to build datadrift_classifier is equal to: 0.7551099716789643


In [95]:
SD.generate_report(
    output_file='analyse_vcub_activite_covid_period.html',
    title_story="Comparatif de l'activité station Vcub entre 2019 et 2020 (année Covid)",
    title_description="""Actvité Data drift 2020 Covid""" # Optional: add a subtitle to describe report
    )

Report saved to ./analyse_vcub_activite_covid_period.html. To upload and share your report, create a free Datapane account by running `!datapane signup`.

# Analyse avant Covid et après Covid

Le but est d'analyser les différences qu'il peut y avoir sur l'activité des stations Vcub avant le Covid et après (à partir de Septembre 2021). 

Il peut y avoir de nombreux changement socio / démographique qui ont eu lieu entre ces 2 moments comme le télétravail, l'achat d'un vélo, déménagement etc...

In [96]:
# Get data from API pour période après Covid.
start_date='2021-09-01' 
stop_date='2022-05-01' 

# list des station vcub
list_station_id = station_profile['station_id'].unique()

# List des station vcub que l'on monitor (avec un activité assez élevé afin de détecter des possibles anomalie)
list_station_id_pred = \
    station_profile[station_profile['mean'] >= THRESHOLD_PROFILE_STATION]['station_id'].unique()


# +/- 3 min de query API
station_json = get_data_from_api_by_station(station_id=list_station_id, 
                                            start_date=start_date,
                                            stop_date=stop_date)

after_covid_df = transform_json_station_data_to_df(station_json)

In [97]:
after_covid_df.head()

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all
0,1,2021-09-09 14:40:00,18.0,14.0,1.0,0.0,0.0,0.0
1,1,2021-09-09 14:50:00,18.0,14.0,1.0,0.0,0.0,0.0
2,1,2021-09-09 15:00:00,18.0,14.0,1.0,0.0,0.0,0.0
3,1,2021-09-09 15:10:00,18.0,14.0,1.0,0.0,0.0,0.0
4,1,2021-09-09 15:20:00,18.0,14.0,1.0,0.0,0.0,0.0


In [99]:
after_covid_df['year_month'] = after_covid_df.date.dt.strftime('%Y-%m')

In [104]:
after_covid_df['year_month'].value_counts()

2021-10    816912
2022-01    812448
2021-12    812448
2021-11    788330
2022-02    566566
2021-09    563640
Name: year_month, dtype: int64

Il y a pas mal de trou de données dans l'API d'Oslandia (absence de données à partir du 22/02/2022). On va donc prendre la même période pour 2019 (du 09/20219 au 22/02/2020)

In [126]:
# Get data from API pour période avant Covid.
start_date='2019-09-01' 
stop_date='2020-02-22' 

# list des station vcub
list_station_id = station_profile['station_id'].unique()

# +/- 3 min de query API
station_json = get_data_from_api_by_station(station_id=list_station_id, 
                                            start_date=start_date,
                                            stop_date=stop_date)

before_covid_df = transform_json_station_data_to_df(station_json)

In [127]:
before_covid_df['year_month'] = before_covid_df.date.dt.strftime('%Y-%m')
before_covid_df['year_month'].value_counts()

2020-01    779889
2019-12    773895
2019-10    757250
2019-11    741376
2019-09    728034
2020-02    533889
Name: year_month, dtype: int64

In [128]:
def process_dataframe_to_analyse(data):
    """
    To not duplicated code 
    
    """
    # Create feature
    data = get_consecutive_no_transactions_out(data)

    # Date format yyyy-mm-dd
    data['date_day'] = data['date'].dt.date


    # transactions_in : on sommes les dépôts de Vcub à la journée
    # transactions_out : on sommes les prise de Vcub à la journée
    # consecutive_no_transactions_out : on prend le maximum de temps (10min) ou aucun prise de Vcub n'a été faite durant la journée.
    data_agg = \
        data.groupby(['date_day', 'station_id'], as_index=False)[['transactions_in', 
                                                                  'transactions_out',
                                                                  'consecutive_no_transactions_out']].agg({'transactions_in': 'sum',
                                                                                                           'transactions_out': 'sum',
                                                                                                           'consecutive_no_transactions_out': 'max'})

    data_agg['date_day'] = data_agg['date_day'].astype(str)
    data_agg['year'] = data_agg['date_day'].apply(lambda x: x[0:4])

    # Création de features temporelle
    data_agg['month'] = pd.to_numeric(data_agg['date_day'].apply(lambda x: x[5:7]))

    return data_agg

before_covid_agg_df = process_dataframe_to_analyse(data=before_covid_df)
after_covid_agg_df = process_dataframe_to_analyse(data=after_covid_df)

In [129]:
before_covid_agg_df.tail()

,date_day,station_id,transactions_in,transactions_out,consecutive_no_transactions_out,year,month
29966,2020-02-21,176,9.0,7.0,59,2020,2
29967,2020-02-21,177,13.0,7.0,13,2020,2
29968,2020-02-21,178,9.0,19.0,54,2020,2
29969,2020-02-21,179,12.0,13.0,45,2020,2
29970,2020-02-21,180,0.0,0.0,188,2020,2


In [130]:
after_covid_agg_df.tail()

,date_day,station_id,transactions_in,transactions_out,consecutive_no_transactions_out,year,month
30457,2022-02-22,180,0.0,0.0,170,2022,2
30458,2022-02-22,181,1.0,5.0,117,2022,2
30459,2022-02-22,182,0.0,0.0,281,2022,2
30460,2022-02-22,183,0.0,0.0,162,2022,2
30461,2022-02-22,251,5.0,5.0,55,2022,2


In [146]:
FEATURE_TO_ANALYSE = ['transactions_in', 'transactions_out', 'consecutive_no_transactions_out', 'station_id', 'month']

# Analyse de l'ensemble des stations Vcub
# SD = SmartDrift(df_current=after_covid_agg_df[FEATURE_TO_ANALYSE],
#                 df_baseline=before_covid_agg_df[FEATURE_TO_ANALYSE],
#                 )

# Analyse sur les stations que l'on monitor (+/- 65 stations)
SD = SmartDrift(df_current=after_covid_agg_df[after_covid_agg_df['station_id'].isin(list_station_id_pred)][FEATURE_TO_ANALYSE],
                df_baseline=before_covid_agg_df[before_covid_agg_df['station_id'].isin(list_station_id_pred)][FEATURE_TO_ANALYSE],
                )

In [145]:
after_covid_agg_df[after_covid_agg_df['station_id'].isin(list_station_id_pred)][FEATURE_TO_ANALYSE]

,transactions_in,transactions_out,consecutive_no_transactions_out,station_id,month
0,13.0,22.0,13,1,9
1,13.0,21.0,10,2,9
2,15.0,17.0,15,3,9
3,19.0,13.0,15,4,9
4,39.0,39.0,6,5,9
...,...,...,...,...,...
30414,6.0,7.0,54,135,2
30415,10.0,11.0,46,136,2
30417,11.0,17.0,30,139,2
30449,22.0,15.0,42,172,2


In [147]:
SD.compile(full_validation=True,
           date_compile_auc = '02/03/2022',       # Optional: useful when computing the drift for a time that is not now
           datadrift_file = "before_and_after_station_pred_covid.csv"    # Optional: name of the csv file that contains the performance history of data drift
            )

The computed AUC on the X_test used to build datadrift_classifier is equal to: 0.7061915204678363
before_and_after_station_pred_covid.csv did not exist and was created. 


In [148]:
SD.generate_report(
    output_file='analyse_vcub_activite_station_pred_before_and_after_covid_period.html',
    title_story="Comparatif de l'activité des stations monitorées Vcub avant (01/09/2019 au 22/02/2020) et après le covid (01/09/2021 au 22/02/2022)",
    title_description="""Activité Vcub drift avant et après Covid""" # Optional: add a subtitle to describe report
    )

Report saved to ./analyse_vcub_activite_station_pred_before_and_after_covid_period.html. To upload and share your report, create a free Datapane account by running `!datapane signup`.

**Conclusion :**

- Sur l'ensemble des stations Vcub, l'AUC classifier Datadrift est de 0.68
- Sur les stations qui sont monitoré par vcub_keeper (les plus actives), l'AUC du classifier Datadrift monte à 0.70


C'est surtout la variable `consecutive_no_transactions_out` qui permet de différencier l'activité avant et après Covid avec en moyenne un temps d'absence de prise de vélo pour une station Vcub plus long après le Covid. On remarque aussi que les `transactions_in` et `transactions_out` (dépôt et prise de Vcub) sont moins fréquents après le Covid qu'avant (redondant avec l'absence cumulée de prise de Vucb `consecutive_no_transactions_out`)

## Analyse par station

In [153]:
#after_covid_agg_df[after_covid_agg_df['station_id'] == 37][FEATURE_TO_ANALYSE]

In [231]:
FEATURE_TO_ANALYSE = ['transactions_in', 'transactions_out', 'consecutive_no_transactions_out','month']
data_dict = []

for station_id in list_station_id_pred:

    df_current =  after_covid_agg_df[after_covid_agg_df['station_id'] == station_id][FEATURE_TO_ANALYSE]
    df_baseline = before_covid_agg_df[before_covid_agg_df['station_id'] == station_id][FEATURE_TO_ANALYSE]

    if (len(df_baseline) <= 50) or (len(df_current) <= 50):
        print("Pas assez de point de comparaison pour la station N°" + str(station_id))
    else:
        SD = SmartDrift(df_current=df_current,
                        df_baseline=df_baseline,
                        )
        SD.compile(full_validation=True)
        print("Station N°"+ str(station_id) + " : " + str(SD.auc))
        data_dict.append({'station_id' : station_id, 'auc': SD.auc})

        # Reporting
        output_file_name = "drift_by_station/analyse_vcub_activite_station_pred_before_and_after_covid_period_station_" +\
            str(station_id) + ".html"
        SD.generate_report(
        output_file=output_file_name,
        title_story="Comparatif de l'activité des stations monitorées Vcub avant (01/09/2019 au 22/02/2020) et après le covid (01/09/2021 au 22/02/2022)",
        title_description="""Activité Vcub drift avant et après Covid""", # Optional: add a subtitle to describe report
        #working_dir="drift_by_station"
        )

Station N°124 : 0.6695652173913044


Report saved to ./drift_by_station/analyse_vcub_activite_station_pred_before_and_after_covid_period_station_124.html. To upload and share your report, create a free Datapane account by running `!datapane signup`.

Station N°15 : 0.6945652173913043


Report saved to ./drift_by_station/analyse_vcub_activite_station_pred_before_and_after_covid_period_station_15.html. To upload and share your report, create a free Datapane account by running `!datapane signup`.

Station N°60 : 0.6978260869565216


...

Report saved to ./drift_by_station/analyse_vcub_activite_station_pred_before_and_after_covid_period_station_123.html. To upload and share your report, create a free Datapane account by running `!datapane signup`.

Station N°39 : 0.7663043478260869


Report saved to ./drift_by_station/analyse_vcub_activite_station_pred_before_and_after_covid_period_station_39.html. To upload and share your report, create a free Datapane account by running `!datapane signup`.

Station N°106 : 0.7521739130434782


Report saved to ./drift_by_station/analyse_vcub_activite_station_pred_before_and_after_covid_period_station_106.html. To upload and share your report, create a free Datapane account by running `!datapane signup`.

In [236]:
drift_by_station_df = pd.DataFrame(data_dict).sort_values('auc', ascending=0)
drift_by_station_df

,station_id,auc
16,136,0.898913
37,41,0.850543
55,103,0.823913
42,1,0.802174
17,21,0.798913
29,37,0.798913
60,123,0.798913
40,55,0.791304
26,19,0.791304
34,7,0.788043


In [242]:
drift_by_station_df[drift_by_station_df['auc'] <= 0.6].shape

(4, 2)